In [1]:
pip install scapy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
 python -m pip install "tensorflow<2.11"

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4060883281.py, line 1)

In [ ]:
from scapy.all import sniff

def packet_handler(packet):
    # Process packet here
    pass

sniff(iface="eth0", prn=packet_handler, store=False)

In [6]:
import pandas as pd
from scapy.all import sniff, IP, TCP, UDP
import time

# 📌 Store extracted data in a list
live_data = []

# 📌 Define packet processing function
def process_packet(packet):
    global live_data
    
    if packet.haslayer(IP):  # Ensure the packet has an IP layer
        row = {
            "timestamp": time.time(),  # Capture timestamp
            "duration": 0,  # Placeholder, computed later
            "protocol_type": packet[IP].proto,  # 6=TCP, 17=UDP, 1=ICMP
            "service": "unknown",  # Placeholder, mapped from ports later
            "flag": "SF" if packet.haslayer(TCP) and "S" in str(packet[TCP].flags) else "OTH",
            "src_bytes": len(packet.payload),
            "dst_bytes": len(packet.payload),  # Approximation
            "land": 1 if packet[IP].src == packet[IP].dst else 0,
            "wrong_fragment": 1 if packet[IP].flags & 1 else 0,
            "urgent": 1 if packet.haslayer(TCP) and packet[TCP].urgptr > 0 else 0,
            "num_failed_logins": 0,  # Requires auth tracking
            "logged_in": 0,  # Requires session tracking
            "num_compromised": 0,  # Placeholder
            "root_shell": 0,  # Not detectable via Scapy alone
            "su_attempted": 0,  # Placeholder
            "num_root": 0,  # Placeholder
            "num_file_creations": 0,  # Placeholder
            "num_shells": 0,  # Placeholder
            "num_access_files": 0,  # Placeholder
            "num_outbound_cmds": 0,  # Placeholder
            "is_host_login": 0,  # Placeholder
            "is_guest_login": 0,  # Placeholder
            "count": 0,  # Placeholder, computed later
            "srv_count": 0,  # Placeholder, computed later
            "serror_rate": 0,  # Placeholder, computed later
            "srv_serror_rate": 0,  # Placeholder, computed later
            "rerror_rate": 0,  # Placeholder, computed later
            "srv_rerror_rate": 0,  # Placeholder, computed later
            "same_srv_rate": 0,  # Placeholder, computed later
            "diff_srv_rate": 0,  # Placeholder, computed later
            "srv_diff_host_rate": 0,  # Placeholder, computed later
            "dst_host_count": 0,  # Placeholder, computed later
            "dst_host_srv_count": 0,  # Placeholder, computed later
            "dst_host_same_srv_rate": 0,  # Placeholder, computed later
            "dst_host_diff_srv_rate": 0,  # Placeholder, computed later
            "dst_host_same_src_port_rate": 0,  # Placeholder, computed later
            "dst_host_srv_diff_host_rate": 0,  # Placeholder, computed later
            "dst_host_serror_rate": 0,  # Placeholder, computed later
            "dst_host_srv_serror_rate": 0,  # Placeholder, computed later
            "dst_host_rerror_rate": 0,  # Placeholder, computed later
            "dst_host_srv_rerror_rate": 0,  # Placeholder, computed later
            "label": "unknown",  # Placeholder for attack/normal
            "difficulty": 0  # Placeholder
        }
        
        # 📌 Assign service based on port numbers
        if packet.haslayer(TCP) or packet.haslayer(UDP):
            port = packet[TCP].dport if packet.haslayer(TCP) else packet[UDP].dport
            common_ports = {80: "http", 443: "https", 21: "ftp", 22: "ssh", 25: "smtp", 53: "dns"}
            row["service"] = common_ports.get(port, "unknown")

        # 📌 Append packet details to live_data
        live_data.append(row)

        # 📌 Convert to Pandas DataFrame
        df = pd.DataFrame(live_data)

        # 📌 Compute session-based statistics dynamically
        df["duration"] = df.groupby(["src_bytes", "dst_bytes"])["timestamp"].transform(lambda x: x.max() - x.min())
        df["count"] = df.groupby("service")["service"].transform("count")
        df["srv_count"] = df.groupby("service")["service"].transform("count")
        df["serror_rate"] = df["flag"].apply(lambda x: 1 if "S" in x else 0).rolling(window=100, min_periods=1).mean()
        df["rerror_rate"] = df["flag"].apply(lambda x: 1 if "R" in x else 0).rolling(window=100, min_periods=1).mean()
        df["same_srv_rate"] = df["service"].map(df["service"].value_counts(normalize=True))
        df["diff_srv_rate"] = 1 - df["same_srv_rate"]

        # 📌 Compute host-based statistics dynamically
        df["dst_host_count"] = df.groupby("dst_bytes")["dst_bytes"].transform("count")
        df["dst_host_srv_count"] = df.groupby(["dst_bytes", "service"])["service"].transform("count")
        df["dst_host_same_srv_rate"] = df["dst_host_srv_count"] / df["dst_host_count"]
        df["dst_host_diff_srv_rate"] = 1 - df["dst_host_same_srv_rate"]

        # 📌 Print extracted features for each packet
        print(df.tail(1))  # Display the latest captured packet features

# 📌 Start sniffing live packets
print("🚀 Capturing live packets... Press Ctrl+C to stop.")
sniff(prn=process_packet, store=False, filter="ip", count=0)  # Capture all IP packets in real-time


🚀 Capturing live packets... Press Ctrl+C to stop.
      timestamp  duration  protocol_type service flag  src_bytes  dst_bytes  \
0  1.740058e+09       0.0              6   https   SF         52         52   

   land  wrong_fragment  urgent  ...  dst_host_same_srv_rate  \
0     0               0       0  ...                     1.0   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                     0.0                            0   

   dst_host_srv_diff_host_rate  dst_host_serror_rate  \
0                            0                     0   

   dst_host_srv_serror_rate  dst_host_rerror_rate  dst_host_srv_rerror_rate  \
0                         0                     0                         0   

     label  difficulty  
0  unknown           0  

[1 rows x 43 columns]
      timestamp  duration  protocol_type  service flag  src_bytes  dst_bytes  \
1  1.740058e+09  0.035284              6  unknown   SF         52         52   

   land  wrong_fragment  urgent  ...  d

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [2]:
from scapy.all import sniff

def packet_callback(packet):
    print(packet.show())

sniff(prn=packet_callback, store=False , count=1) 

###[ Ethernet ]###
  dst       = da:cd:a2:00:1f:96
  src       = 38:d5:7a:8a:dc:2d
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 153
     id        = 15934
     flags     = DF
     frag      = 0
     ttl       = 128
     proto     = tcp
     chksum    = 0x1b9c
     src       = 192.168.30.26
     dst       = 20.189.173.5
     \options   \
###[ TCP ]###
        sport     = 52578
        dport     = https
        seq       = 555774555
        ack       = 133661604
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 253
        chksum    = 0x711e
        urgptr    = 0
        options   = []
###[ Raw ]###
           load      = b'\x17\x03\x03\x00l\xb9Tfj\xe7\x04q\xf3\xe00\xdb|\xf4\x03\xd5b$\xdck^\xf4\xc4\xa2\x91\xbbxiL/\xa8\r\xbd\x071\xde\x90\xca\xa7\xd3\x98z\xab\x18\xfeB=\x1c\x15~\xba\xc2\xcc\\\x80\xbeN>\xf39\xfe\xc71\x99\x8d\x8b\xc1\xe7\xd4i!\xb7H\x98!\x95\xe9:\\\xb6L+\xdf\xe0u*(Y\t:`>K|A\x9a\

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [8]:
from scapy.all import get_if_list

print(get_if_list())  # Shows all available interfaces


['\\Device\\NPF_{37512F5B-0591-4E8E-A256-2A480CBB19CD}', '\\Device\\NPF_{0C2CB254-2602-47E8-B947-8583E3498632}', '\\Device\\NPF_{C632B206-97C7-4C12-A8B7-566A03836F69}', '\\Device\\NPF_{5315327A-0FB3-4DB6-B7F7-EEE0CC7CE7D2}', '\\Device\\NPF_{9BC02136-3E0D-4782-9288-7D382CD1C2D4}', '\\Device\\NPF_{1AD98CF5-5CB3-48F5-8862-7C9A4124E9AA}', '\\Device\\NPF_{D1F728C2-3A82-4217-A730-09C469D5FD8D}', '\\Device\\NPF_Loopback', '\\Device\\NPF_{CD50F777-6FD8-406C-ABDA-8CD5607107BF}', '\\Device\\NPF_{21B2B91F-5975-4E3A-9233-3A0451F0E58C}', '\\Device\\NPF_{9A311400-BB12-4E30-A2DB-2ABB7BE8250F}']


In [10]:
from scapy.all import conf, sniff

def packet_handler(packet):
    print(packet.summary())

sniff(iface=conf.iface, prn=packet_handler, store=False , count=5)


Ether / IP / TCP 34.211.82.50:https > 192.168.30.26:50713 A / Padding
Ether / IP / TCP 34.211.82.50:https > 192.168.30.26:50713 A / Padding
Ether / IP / TCP 34.211.82.50:https > 192.168.30.26:50713 PA / Raw
Ether / IP / TCP 34.211.82.50:https > 192.168.30.26:50713 PA / Raw
Ether / IP / TCP 192.168.30.26:50713 > 34.211.82.50:https A


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>